## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [3]:
import glob

import numpy as np
import scipy as sp
import scipy.signal
import scipy.io
import matplotlib.pyplot as plt

# Constants for bandpass filtering
LOW_BPM = 40
HIGH_BPM = 220

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def BandpassFilter(sig, freq_filter=(LOW_BPM / 60, HIGH_BPM / 60), fs=125):
    """Bandpass Filter.

    Args:
        signal: (np.array) The input signal
        pass_band: (tuple) The pass band
        Frequency components outside two elements in the tuple will be removed
        fs: (number) The sampling rate of <signal>

    Returns:
        (np.array) The filtered signal
    """
    b, a = scipy.signal.butter(3, freq_filter, btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, sig)


def ProcessSignal(sig, fs):
    """
    Takes the fourier transform of the signal within the frequency filter window

    Args:
        signal: (np.array) The input signal
        fs: (number) The sampling rate of <signal>

    Returns:
        (np.array) The fourier transform of the signal
        (np.array) Signal frequencies
    """
    
    freqs = np.fft.rfftfreq(len(sig), 1/fs)
    fft = np.abs(np.fft.rfft(sig))
    
    fft[freqs <= LOW_BPM/60.0] = 0.0
    fft[freqs >= HIGH_BPM/60.0] = 0.0
    
    return fft, freqs

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Algorithm to estimate heart rate and return an error and confidence value for each estimate. Error is with respect to reference heart rate.
    The algorithm:
    (1) Processes the ppg signal through a bandpass filter
    (2) Aggregates the 3 channel accelerometer signal into a single acc signal and processes it through the bandpass filter
    (3) For each 8-second sliding window with a 6-second overlap between successive windows:
        (i) Takes the fourier transforms of the ppg and acc signals and sorts the frequencies in descending order of the FFT magnitudes.
        This gives us the dominant frequencies for the ppg and accelerometer signals. 
        (ii) If the dominant ppg frequency is greater than the dominant accelerometer frequency by at least 0.1,
        take the dominant ppg frequency as the heart rate estimate/second.
        (iii) Else if the next dominant ppg frequency is not in the top 5 dominant acc frequencies,
        take this ppg frequency as the heart rate estimate/second.
        (iv) Else if the next dominant ppg frequency is not in the top 5 dominant acc frequencies,
        take this ppg frequency as the heart rate estimate/second.
        (v) If none of the above conditions are true, then simply take the most dominant ppg frequency as the heart rate estimate.
        (vi) Compute the confidence by totalling the fft magnitudes for frequencies near the above estimate and
        dividing it by the sum of the entire magnitude of frequencies for the ppg signal.
        (vii) Compute error by taking the difference between the heart rate estimate and reference value for the same window.
     
    Usage:
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)

    Args:
        data_fl: (str) filepath to a troika .mat file containing ppg and acc signals.
        ref_fl: (str) filepath to the reference or "ground truth" heart rate corresponding to the data_fl

    Returns:
        2-D numpy arrays for errors and confidence. 
        Each entry represents an error and confidence value for an 8-second signal window with 6-second overlap between successive windows.
    """
    
    
    # Initialize constants
    fs = 125
    pks = []
    pks_bpm = []
    errors = []
    confidence = []
    
    window_length = fs * 8   # 8 second window
    window_shift = fs * 2   # 2 second outputs
    window_confidence = 5/60   # frequency window around peak frequency (estimated heart rate) to calculate signal power 
    
    # Load signals
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ppg = BandpassFilter(ppg)
    
    # Aggregate the accelerometer signal. Since the y-channel measures gravity, we normalize it
    acc = np.sqrt(accx**2 + (accy - np.mean(accy))**2 + accz**2)
    acc = BandpassFilter(acc)
        
    ref_pks = sp.io.loadmat(ref_fl)['BPM0']
    for i in range(0, len(ppg) - window_length, window_shift):
        ppg_window = ppg[i:i + window_length]
        acc_window = acc[i:i + window_length]

        fft_ppg, freqs_ppg = ProcessSignal(ppg_window, fs)
    
        # Sort for dominant ppg frequencies 
        order = np.argsort(np.abs(fft_ppg))[::-1]
        most_imp_freqs_ppg = list(freqs_ppg[order])[:5]
    
        fft_acc, freqs_acc = ProcessSignal(acc_window, fs)
    
        # Sort for dominant acc frequencies
        order = np.argsort(np.abs(fft_acc))[::-1]
        most_imp_freqs_acc = list(freqs_acc[order])[:5]
    
        # Heuristic to estimate heart rate from ppg and acc dominant frequencies
        threshold = 0.1
        if (most_imp_freqs_ppg[0] - most_imp_freqs_acc[0] > threshold):
            peak_freq = most_imp_freqs_ppg[0]
        elif (most_imp_freqs_ppg[1] not in most_imp_freqs_acc):
            peak_freq = most_imp_freqs_ppg[1]
        elif (most_imp_freqs_ppg[2] not in most_imp_freqs_acc):
            peak_freq = most_imp_freqs_ppg[2]
        else:
            peak_freq = most_imp_freqs_acc[0]
            
        pks.append(peak_freq)
        
        # Find confidence
        fundamental_frequency_window = (freqs_ppg > peak_freq - window_confidence) & (freqs_ppg > peak_freq + window_confidence)
        ppg_power = np.sum(fft_ppg[fundamental_frequency_window])
        total_power = np.sum(fft_ppg)
        conf = ppg_power / total_power
        confidence.append(conf)
    
    # Trim the estimated peaks and confidence array to the size of reference array
    ref_fl_len = len(ref_pks)
    pks = pks[:ref_fl_len]
    confidence = confidence[:ref_fl_len]
    
    # Convert cycles/sec to cycles/min or bpm 
    pks_bpm = [element * 60 for element in pks]
    
    # Compute element-wise error between estimated and reference heart rates
    zip_object = zip(pks_bpm, ref_pks)
    for pks_i, ref_i in zip_object:
        error = np.abs(pks_i-ref_i)[0]
        errors.append(error)

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    return errors, confidence

def main():
    Evaluate()
    
if __name__ == "__main__":
    main()    
    

14.5361091598


-----
### Project Write-up

> - **Code Description** - The code implements an algorithm to estimate the pulse rate for a subject from a given set of ppg and 3-channel accelerometer signals provided for 12 subjects. Specifically, for each subject, we are given one ppg signal and one signal each for the x, y and z channels of the accelerometer. The code fine tunes the heuristic algorithm based on its error with respect to the reference signal given in beats per minute. The code assumes that the pulse rate is restricted to between 40 and 220 bpm. In addition, the code produces a confidence for each estimate. It produces a pulse estimate and confidence output every 2 seconds based on a moving window of 8 seconds of signal, with a 6 second overlap between successive windows, which is how the reference data is specified. The data comes from the Troika dataset (see references), which contains the signal and reference data in .mat format. The code uses scipy.io.loadmat to load these data into Python.

> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
The data for this project comes from the Troika dataset, with only the PPG and 3-channel accelerometer data retained, and ECG signal discarded, and made available in the .mat format. Per the data description, "It is recorded from 12 subjects with age from 18 to 35 during fast running at the peak speed of 15 km/h. For each subject, the PPG signals were recorded from wrist by two pulse oximeters with green LEDs (wavelength: 515nm). Their distance (from center to center) was 2 cm. The acceleration signal was also recorded from wrist by a three-axis accelerometer. Both the pulse oximeter and the accelerometer were embedded in a wristband, which was comfortably worn...All signals were sampled at 125 Hz and sent to a nearby computer via Bluetooth". The signal data files are prefixed with DATA_ while the reference files are prefixed with REF_. The data files have the naming convention: DATA_<subjectID>_<TYPE> where TYPE is TYPE01 or TYPE02 based on how the running speeds changed for participants on the treadmill during the recording. The ground truth files follow the same naming convention, namely REF_<subjectID>_<TYPE>, with one reference file corresponding to each signal file. The reference file contains a variable BPM0 which contains the reference signal, with each entry representing an 8-second signal with a 6-second overlap between successive entries. The data is limited in that it contains a small number of subjects with two types of running schedules only. A more complete real-world dataset is likely to be more noisy, contain an ECG signal and more noise and a larger variety of frequencies in the signals, based on normal human activities such as sitting, standing, reclining, several different types of physical activity and sleep. The test dataset is an unseen dataset, made available for black box testing only. 

> - **Algorithm Description** - 
The algorithm works in the following way:
1 - Aggregates the 3-channel accelerometer signal into a single "acc" signal which is the L2 norm of the 3 channels - square root of the sum of squares of the accx, accy and accz. However, given that the accelerometer y-channel measures gravity constantly, we normalize accy with respect to its mean. In other words, accy = accy - np.mean(accy)
2 - Filters the ppg and consolidated acc signal using a bandpass filter which restricts the signal to between 40 and 220 bpm.
3 - Iterates through 8-second moving windows with a 6-second overlap, doing the following processing for each window:
    (i) Takes the fourier transforms of the ppg and acc signals and sorts the frequencies in descending order of the FFT magnitudes. This gives us the dominant frequencies for the ppg and accelerometer signals.
    (ii) If the dominant ppg frequency is greater than the dominant accelerometer frequency by at least 0.1, take the dominant ppg frequency as the heart rate estimate/second.
    (iii) Else if the next dominant ppg frequency is not in the top 5 dominant acc frequencies, take this ppg frequency as the heart rate estimate/second
    (iv) Else if the next dominant ppg frequency is not in the top 5 dominant acc frequencies, take this ppg frequency as the heart rate estimate/second
    (v) If none of the above conditions are true, then simply take the most dominant ppg frequency as the heart rate estimate.
    (vi) Compute the confidence by totalling the fft magnitudes for frequencies near the above estimate and dividing it by the sum of the entire magnitude of frequencies for the ppg signal.
    (vii) Compute error by taking the difference between the heart rate estimate and reference value for the same window. 
The ppg measures the pulse rate by measuring the strength of signal as the heart contracts and expands. The accelerometer signal measures the signal based on the physical movement of the subject as they exercise. One limitiation of the algorithm is that the accelerometer frequency may sometimes be confused with the ppg signal, and it may be difficult to distinguish the  two, for example, if the cadence of the arm movement is in line with the heart beating. The ppg signal is also likely to be affected by noise and may pick up other frequencies beyond the heart rate.

> - **Algorithm Performance** - The algorithm is a heuristic one and does not employ machine learning. The algorithm performance is evaluated using the Mean Absolute Error (MAE) of the best 90% of the estimates. We get a MAE of 12.23 on the unseen test dataset. In order to reduce error and generalize on different datasets, we might employ a machine learning algorithm such as Random Forest Regressor. The features used might include the mean, standard deviation and 5th, 10th, 25th, 50th and 90th percentiles of the ppg and each of the accelerometer signals. We may also caution users of the algorithm to consider the effects of noise such as melanin (affects ppg signal) and arm movement, arm position and finger movement (affects acc signal), as well as rapidly changing signals within a small time frame, all of which can affect the performance numbers.

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 